## Analyze whether SNWD varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [4]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='SSSBSBBB'
meas='SNWD'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
526
+------------------+-------------------+-----------------+---------+--------+--------+---------+-----------+-------------------+-------------------+-------------------+------------------+-----------+------------+------+--------------------+------+
|           coeff_1|            coeff_2|          coeff_3|elevation|   label|latitude|longitude|measurement|              res_1|              res_2|              res_3|          res_mean|    station|   total_var|undefs|              vector|  year|
+------------------+-------------------+-----------------+---------+--------+--------+---------+-----------+-------------------+-------------------+-------------------+------------------+-----------+------------+------+--------------------+------+
|-6615.426188381523|-1465.9875077165289|1026.709373492892|   1809.6|SSSBSBBB| 39.3239|-120.2331|       SNWD|0.20641434500585848|0.16362354251904537|0.14452109933445303|0.654001970533

In [5]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [6]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_3'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_3 FROM weather


station    year      coeff_3
0  USC00042467  1998.0  1026.709373
1  USC00048758  1998.0   797.125516
2  USC00041018  1950.0  1726.007565
3  USC00049043  1978.0   843.754024
4  USC00048474  1973.0 -3107.030252

In [7]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_3')
year_station_table.head(10)

station  US1CAPC0001  USC00040747  USC00040931  USC00041018  USC00041462  \
year                                                                       
1900.0           NaN          NaN          NaN          NaN          NaN   
1901.0           NaN          NaN          NaN          NaN          NaN   
1902.0           NaN          NaN          NaN          NaN          NaN   
1903.0           NaN          NaN          NaN          NaN          NaN   
1904.0           NaN          NaN          NaN          NaN          NaN   
1906.0           NaN          NaN          NaN          NaN          NaN   
1922.0           NaN          NaN          NaN          NaN          NaN   
1932.0           NaN          NaN          NaN          NaN          NaN   
1933.0           NaN          NaN          NaN          NaN          NaN   
1937.0           NaN          NaN          NaN          NaN          NaN   

station  USC00042334  USC00042338  USC00042467  USC00042500  USC00044288  \
year                                                                       
1900.0           NaN          NaN          NaN          NaN          NaN   
1901.0           NaN          NaN          NaN          NaN          NaN   
1902.0           NaN          NaN          NaN          NaN          NaN   
1903.0           NaN          NaN          NaN          NaN          NaN   
1904.0           NaN          NaN          NaN          NaN          NaN   
1906.0           NaN          NaN          NaN          NaN          NaN   
1922.0           NaN          NaN          NaN          NaN          NaN   
1932.0           NaN          NaN          NaN          NaN          NaN   
1933.0           NaN          NaN          NaN          NaN          NaN   
1937.0           NaN          NaN          NaN          NaN          NaN   

station     ...       USC00260507  USC00261485  USC00262708  USC00264858  \
year        ...                                                            
1900.0      ...               NaN          NaN          NaN          NaN   
1901.0      ...               NaN          NaN          NaN          NaN   
1902.0      ...               NaN          NaN          NaN          NaN   
1903.0      ...               NaN          NaN          NaN          NaN   
1904.0      ...               NaN          NaN          NaN          NaN   
1906.0      ...               NaN          NaN          NaN          NaN   
1922.0      ...               NaN          NaN    431.01239          NaN   
1932.0      ...        599.637725          NaN          NaN          NaN   
1933.0      ...        327.810018          NaN          NaN          NaN   
1937.0      ...               NaN          NaN          NaN          NaN   

station  USC00265440  USC00265447  USC00266791  USC00267820  USC00268761  \
year                                                                       
1900.0           NaN          NaN          NaN          NaN          NaN   
1901.0           NaN          NaN          NaN          NaN          NaN   
1902.0           NaN          NaN          NaN          NaN          NaN   
1903.0           NaN          NaN          NaN          NaN          NaN   
1904.0           NaN          NaN          NaN          NaN          NaN   
1906.0           NaN          NaN          NaN          NaN          NaN   
1922.0           NaN          NaN          NaN          NaN          NaN   
1932.0           NaN          NaN          NaN          NaN          NaN   
1933.0           NaN          NaN          NaN          NaN          NaN   
1937.0           NaN          NaN          NaN          NaN          NaN   

station  USW00023225  
year                  
1900.0           NaN  
1901.0           NaN  
1902.0           NaN  
1903.0           NaN  
1904.0           NaN  
1906.0           NaN  
1922.0           NaN  
1932.0           NaN  
1933.0           NaN  
1937.0           NaN  

[10 rows x 34 columns]

### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [8]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station)
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)

total RMS                   =  1413.02162495
RMS removing mean-by-station=  1339.19817565
RMS removing mean-by-year   =  679.275736328


In [9]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 1413.02162495
0 after removing mean by year    = 679.275736328
0 after removing mean by stations= 637.394901309
1 after removing mean by year    = 631.398747744
1 after removing mean by stations= 630.292198974
2 after removing mean by year    = 630.042122608
2 after removing mean by stations= 629.972336213
3 after removing mean by year    = 629.948321363
3 after removing mean by stations= 629.938464985
4 after removing mean by year    = 629.933880282
4 after removing mean by stations= 629.931571687
